# 1.6 套利定价理论 (Arbitrage Pricing Theory, APT)

## 概念介绍

套利定价理论(APT)是由Stephen Ross于1976年提出的一种多因子资产定价模型，是对资本资产定价模型(CAPM)的扩展。与CAPM只考虑市场风险不同，APT认为资产回报率受多种系统性风险因子影响。

### APT模型公式

套利定价理论的基本方程：

$$E(R_i) = R_f + \beta_{i1}RP_1 + \beta_{i2}RP_2 + ... + \beta_{in}RP_n$$

其中：
- $E(R_i)$ 是资产$i$的期望收益率
- $R_f$ 是无风险收益率
- $\beta_{ij}$ 是资产$i$对因子$j$的敏感度
- $RP_j$ 是因子$j$的风险溢价

### APT与CAPM的区别

- **多因子模型**：APT允许多个因子影响资产收益率，而CAPM只考虑市场风险一个因子
- **更少的假设**：APT比CAPM有更少的限制性假设，更适合现实市场
- **因子选择灵活**：APT允许使用各种宏观经济因子，如GDP增长率、通货膨胀率、利率等

在这个笔记本中，我们将使用宏观经济因子构建APT模型，分析这些因子如何影响股票收益率。

## 1.1 安装必要的库

首先安装需要用到的Python库

In [ ]:
# 安装必要的库
!pip install pandas numpy matplotlib statsmodels seaborn scikit-learn

## 1.2 从Github上读取数据

In [ ]:
# 从GitHub下载数据文件 - 使用正确的文件名和路径
!wget -O AAPL_data.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_AAPL_data.csv"
!wget -O SPY_data.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_market_data.csv"
!wget -O MSFT_data.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_MSFT_data.csv"
!wget -O GOOGL_data.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_GOOGL_data.csv"
!wget -O risk_free_rate.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_risk_free_rate.csv"
!wget -O unemployment_rate.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_unemployment_rate.csv"
!wget -O inflation_rate.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_inflation_rate.csv"
!wget -O gdp_data.csv "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%201/1.6_gdp_data.csv"

# 设置数据文件路径
file_path_msft = "MSFT_data.csv"
file_path_aapl = "AAPL_data.csv"
file_path_googl = "GOOGL_data.csv"
file_path_market = "SPY_data.csv"  # 市场数据
file_path_rf =  "risk_free_rate.csv"  # 无风险利率
file_path_unrate =  "unemployment_rate.csv"  # 失业率
file_path_cpi = "inflation_rate.csv"  # 通胀率
file_path_gdp =  "gdp_data.csv"  # GDP数据

## 1.3 导入必要的库

In [35]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

## 1.4 数据加载和预处理

In [ ]:
# 加载数据集
try:
    msft_data_df = pd.read_csv('MSFT_data.csv', skiprows=2)
    aapl_data_df = pd.read_csv('AAPL_data.csv', skiprows=2)
    googl_data_df = pd.read_csv('GOOGL_data.csv', skiprows=2)
    market_data_df = pd.read_csv('SPY_data.csv', skiprows=2)
    risk_free_rate_df = pd.read_csv('risk_free_rate.csv')
    unemployment_rate_df = pd.read_csv('unemployment_rate.csv')
    inflation_rate_df = pd.read_csv('inflation_rate.csv')
    gdp_data_df = pd.read_csv('gdp_data.csv')
    print("数据加载成功")
except Exception as e:
    print(f"数据加载失败: {e}")

## 1.5 数据预览和结构检查

In [ ]:
# 查看数据结构
print("MSFT数据前5行:")
display(msft_data_df.head())

print("\n无风险利率数据前5行:")
display(risk_free_rate_df.head())

print("\n失业率数据前5行:")
display(unemployment_rate_df.head())

## 1.6 数据清洗和准备

In [ ]:
# 统一列名
column_names = ["Date", "Price", "Close", "High", "Low", "Open"]  # 移除 'Volume' 列名
msft_data_df.columns = column_names
aapl_data_df.columns = column_names
googl_data_df.columns = column_names
market_data_df.columns = column_names

# 将 "Close" 列重命名为 "Adj Close" 并转换日期列
msft_data_df.rename(columns={'Close': 'Adj Close'}, inplace=True)
aapl_data_df.rename(columns={'Close': 'Adj Close'}, inplace=True)
googl_data_df.rename(columns={'Close': 'Adj Close'}, inplace=True)
market_data_df.rename(columns={'Close': 'Adj Close'}, inplace=True)

In [ ]:
# 转换日期列为日期格式
msft_data_df['Date'] = pd.to_datetime(msft_data_df['Date'])
aapl_data_df['Date'] = pd.to_datetime(aapl_data_df['Date'])
googl_data_df['Date'] = pd.to_datetime(googl_data_df['Date'])
market_data_df['Date'] = pd.to_datetime(market_data_df['Date'])

# 确保所有宏观经济数据的 'Date' 列转换为 datetime 类型
risk_free_rate_df['DATE'] = pd.to_datetime(risk_free_rate_df['DATE'])
unemployment_rate_df['DATE'] = pd.to_datetime(unemployment_rate_df['DATE'])
inflation_rate_df['DATE'] = pd.to_datetime(inflation_rate_df['DATE'])
gdp_data_df['DATE'] = pd.to_datetime(gdp_data_df['DATE'])

In [ ]:
# 将 "Date" 设置为索引
msft_data_df.set_index('Date', inplace=True)
aapl_data_df.set_index('Date', inplace=True)
googl_data_df.set_index('Date', inplace=True)
market_data_df.set_index('Date', inplace=True)

## 1.7 计算收益率

In [ ]:
# 计算每日回报率
msft_data_df['Return'] = msft_data_df['Adj Close'].pct_change()
aapl_data_df['Return'] = aapl_data_df['Adj Close'].pct_change()
googl_data_df['Return'] = googl_data_df['Adj Close'].pct_change()
market_data_df['Return'] = market_data_df['Adj Close'].pct_change()

# 查看计算得到的回报率
print("MSFT回报率前5行:")
display(msft_data_df['Return'].head())

## 1.8 合并数据集

In [ ]:
# 合并数据
merged_df = msft_data_df[['Return']].merge(aapl_data_df[['Return']], on='Date', suffixes=('_MSFT', '_AAPL'))
merged_df = merged_df.merge(googl_data_df[['Return']], on='Date')
merged_df = merged_df.merge(market_data_df[['Return']], on='Date', suffixes=('', '_Market'))

# 重置索引，将 'Date' 列恢复为普通列
merged_df.reset_index(inplace=True)

# 合并宏观经济数据
merged_df = merged_df.merge(risk_free_rate_df[['DATE', 'GS10']], left_on='Date', right_on='DATE', how='left', suffixes=('', '_RiskFree'))
merged_df = merged_df.merge(unemployment_rate_df[['DATE', 'UNRATE']], left_on='Date', right_on='DATE', how='left', suffixes=('', '_Unemployment'))
merged_df = merged_df.merge(inflation_rate_df[['DATE', 'CPIAUCSL']], left_on='Date', right_on='DATE', how='left', suffixes=('', '_Inflation'))
merged_df = merged_df.merge(gdp_data_df[['DATE', 'GDP']], left_on='Date', right_on='DATE', how='left', suffixes=('', '_GDP'))

# 删除重复的 'DATE' 列
merged_df = merged_df.drop(columns=['DATE'])

# 删除缺失值
merged_df = merged_df.dropna()

## 1.9 计算超额收益

In [ ]:
# 计算超额收益
merged_df['Excess_Return_MSFT'] = merged_df['Return_MSFT'] - merged_df['GS10'] / 100
merged_df['Excess_Return_AAPL'] = merged_df['Return_AAPL'] - merged_df['GS10'] / 100
merged_df['Excess_Return_GOOGL'] = merged_df['Return'] - merged_df['GS10'] / 100

# 查看超额收益数据
print("合并数据集前5行:")
display(merged_df.head())

## 1.10 建立APT模型

In [ ]:
# 设置回归的自变量（宏观经济因子）
factors = merged_df[['GS10', 'UNRATE', 'CPIAUCSL', 'GDP']]
factors = sm.add_constant(factors)  # 加入常数项

# 因变量：超额回报
y_msft = merged_df['Excess_Return_MSFT']
y_aapl = merged_df['Excess_Return_AAPL']
y_googl = merged_df['Excess_Return_GOOGL']

# 进行回归分析
model_msft = sm.OLS(y_msft, factors).fit()
model_aapl = sm.OLS(y_aapl, factors).fit()
model_googl = sm.OLS(y_googl, factors).fit()

## 1.11 分析回归结果

In [ ]:
# 输出回归结果
print("MSFT回归结果：")
display(model_msft.summary())

print("\nAAPL回归结果：")
display(model_aapl.summary())

print("\nGOOGL回归结果：")
display(model_googl.summary())

## 1.12 结论

通过APT模型分析，我们可以观察到：

1. 不同股票对宏观经济因子的敏感度各不相同
2. 某些因子对股票收益率有显著影响，而其他因子影响较小
3. 这种多因子模型比单因子CAPM模型能更好地解释资产收益率的变化

这些结果表明，投资者在进行投资决策时，应该考虑多种宏观经济因素对不同资产的影响，而不仅仅是市场风险一个因素。